# 연습문제 2 - 쇼어 알고리즘
## 역사적 배경

컴퓨팅에서는 알고리즘이 입력 문제의 크기에 따라 얼마나 복잡해지는지를 보고 알고리즘의 성능을 측정하곤 합니다. 예를 들어, 덧셈은 더하는 숫자들의 크기에 따라 선형적으로 커지는 알고리즘을 가지고 있습니다. 몇몇 컴퓨팅 문제들에 대한 가장 효율적인 알고리즘들은 입력의 크기에 따라 _지수적으로_ 커지는데, 이는 그 알고리즘들에 대한 비교적 크지 않은 사이즈의 입력도 현재 지구상의 어떤 컴퓨터에서든 해결되기에는 너무 크다는 것을 의미합니다. 우리는 이 사실을 전적으로 확신하기 때문에 대부분의 인터넷 보안도 몇몇 문제들의 해결 불가능함에 의존하고 있습니다.

1994년에 피터 쇼어는 양자 컴퓨터에서 효율적으로 소인수분해하는 것이 가능하다는 걸 보여주었습니다.[1] 이건 매우 놀라운 발견인데, 현재 가장 효율적인 고전 소인수분해 알고리즘은 지수적으로 커지는 알고리즘들 중 하나이기 때문입니다. 사실, [RSA 암호화는](https://en.wikipedia.org/wiki/RSA_(cryptosystem)) 충분히 큰 숫자들을 소인수분해하는 것이 현재 불가능하다는 것에 의존하고 있습니다. 현재 고전 컴퓨터에서 소인수분해하기에 너무 큰 정수들을 양자 컴퓨터에서 소인수분해하려면 몇백만개의 큐비트들과 게이트들을 필요로 하기 때문에, 현재의 양자 컴퓨터에서 성공적으로 실행되기엔 너무 큰 회로를 필요로 합니다.

그래서 Lieven M.K. Vandersypen, Matthias Steffen, Gregory Breyta, Costantino S. Yannoni, Mark H. Sherwood, 그리고 Isaac L. Chuang은 어떻게 한참 전인 2001년에 성공적으로 양자 컴퓨터에서 15를 소인수분해 할 수 있었을까?![2]

쇼어 알고리즘을 위한 회로를 만들 때 가장 어려운 부분은 제어된 $ay \bmod N$을 계산하는 회로를 만드는 부분이다. 이 회로를 다항 수의 게이트들로 만드는 방법은 이미 알려져 있지만, 이는 오늘날의 컴퓨터에게는 아직 너무 크다. 다행히도 만약 문제에 대한 부분적인 정보를 선험적으로 알 수 있다면, 우리는 약간의 '속임수'를 써 더 효율적인 회로를 만들 수 있다.

위에 나온 논문의 저자들은 이 회로를 그들이 사용 가능한 하드웨어에서 실행하기 위해 $7y \bmod 15$를 수행하는 아주 간단한 회로를 발견하였다. 이는 전체 회로를 그들의 하드웨어에서 실행될 수 있을 만큼 작게 만들었다. 이 연습 문제에서는 쇼어 알고리즘에서 사용될 수 있고 `ibmq_santiago`에서 실행될 수 있는 $35y \bmod N$을 수행하는 회로를 만들어볼 것이다.

이 연습문제에서 무슨 일이 일어나는지를 이해하고 싶다면, [쇼어 알고리즘에 대한 키스킷 교과서 페이지를](https://qiskit.org/textbook/ch-algorithms/shor.html) 읽어 보면 도움이 될 것이다. 하지만 꼭 이 페이지를 읽지 않더라도 연습 문제를 풀 수 있을 것이다.

### 참고자료
1. Shor, Peter W. "Algorithms for quantum computation: discrete logarithms and factoring." Proceedings 35th annual symposium on foundations of computer science. Ieee, 1994.
1. Vandersypen, Lieven MK, et al. "Experimental realization of Shor's quantum factoring algorithm using nuclear magnetic resonance." Nature 414.6866 (2001): 883-887.

## 요약: 쇼어 알고리즘

There is an algorithm called [_quantum phase estimation_](https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html) that tells us the phase a gate introduces to a certain type of state. For example, inputs to phase estimation algorithm could be the state $|1\rangle$ and the gate $Z$. If the $Z$-gate acts on the state $|1\rangle$, we get back the same state with an added global phase of $\pi$:

$$
Z|1\rangle = -|1\rangle = e^{i\pi} |1\rangle
$$

And the quantum phase estimation algorithm could work this out for us. You can see another example [here](https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html#2.-Example:-T-gate-).

Shor showed that if we do phase estimation on a gate, $U$, that has the behavior $U|y\rangle = |a y\bmod N\rangle$, we can quickly get some information about $N$’s factors.

## 문제

In this exercise, we will factor 35 by doing phase estimation on a circuit that implements $13y \bmod 35$. The exercise is to create a circuit that does this, and is also small enough to run on `ibmq_santiago`! This is not an easy task, so the first thing we’re going to do is cheat.

A detail of Shor’s algorithm is that our circuit only needs to work on states we can reach through applying $U$ to the starting state $|1\rangle$. I.e. we can use _any_ circuit that has the behavior: 

$$
\begin{aligned}
U|1\rangle &= |13\rangle \\
UU|1\rangle &= |29\rangle \\
UUU|1\rangle &= |27\rangle \\
UUUU|1\rangle &= |1\rangle \\
\end{aligned}
$$

So how can we make this easier for us? Since we only need to correctly transform 4 different states, we can encode these onto two qubits. For this exercise, we will choose to map the 2-qubit computational basis states to the numbers like so:

$$
\begin{aligned}
|1\rangle &\rightarrow |00\rangle \\
|13\rangle &\rightarrow |01\rangle \\
|29\rangle &\rightarrow |10\rangle \\
|27\rangle &\rightarrow |11\rangle \\
\end{aligned}
$$

Why is this “cheating”? Well, to take advantage of this optimization, we need to know all the states $U$ is going to affect, which means we have to compute $ay \bmod N$ until we get back to 1 again, and that means we know the period of $a^x \bmod N$ and can therefore get the factors of $N$. Any optimization like this, in which we use information that would tell us the value $r$, is obviously not going to scale to problems that classical computers can’t solve. 

But the purpose of this exercise is just to verify that Shor’s algorithm does in fact work as intended, and we’re not going to worry about the fact that we cheated to get a circuit for $U$.

<div id='u-definition'></div>
<div class="alert alert-block alert-success">

**Exercise 2a:** Create a circuit ($U$) that performs the transformation:

$$
\begin{aligned}
U|00\rangle &= |01\rangle \\
U|01\rangle &= |10\rangle \\
U|10\rangle &= |11\rangle \\
U|11\rangle &= |00\rangle \\
\end{aligned}
$$

and is controlled by another qubit. The circuit will act on a 2-qubit target register named 'target', and be controlled by another single-qubit register named 'control'. You should assign your finished circuit to the variable '`cu`'.
    
</div>

In [ ]:
from qiskit import QuantumCircuit
from qiskit import QuantumRegister, QuantumCircuit
c = QuantumRegister(1, 'control')
t = QuantumRegister(2, 'target')
cu = QuantumCircuit(c, t, name="Controlled 13^x mod 35")

# 이 선들 사이에 코드를 적으십시오 - 시작





# 이 선들 사이에 코드를 적으십시오 - 끝

cu.draw('mpl')

And run the cell below to check your answer:

In [ ]:
# Check your answer using following code
from qc_grader import grade_ex2a
grade_ex2a(cu)

Congratulations! You’ve completed the hard part. 

We read the output of the phase estimation algorithm by measuring qubits, so we will need to make sure our 'counting' register contains enough qubits to read off $r$. In our case, $r = 4$, which means we only need $\log_2(4) = 2$ qubits (cheating again because we know $r$ beforehand), but since Santiago has 5 qubits, and we've only used 2 for the 'target' register, we'll use all remaining 3 qubits as our counting register.

To do phase estimation on $U$, we need to create circuits that perform $U^{2^x}$ ($U$ repeated $2^x$ times) for each qubit (with index $x$) in our register of $n$ counting qubits. In our case this means we need three circuits that implement:

$$ U, \; U^2, \; \text{and} \; U^4 $$

So the next step is to create a circuit that performs $U^2$ (i.e. a circuit equivalent to applying $U$ twice).

<div class="alert alert-block alert-success">

**Exercise 2b:** Create a circuit ($U^2$) that performs the transformation:

$$
\begin{aligned}
U|00\rangle &= |10\rangle \\
U|01\rangle &= |11\rangle \\
U|10\rangle &= |00\rangle \\
U|11\rangle &= |01\rangle \\
\end{aligned}
$$

and is controlled by another qubit. The circuit will act on a 2-qubit target register named 'target', and be controlled by another single-qubit register named 'control'. You should assign your finished circuit to the variable '`cu2`'.
</div>

In [ ]:
c = QuantumRegister(1, 'control')
t = QuantumRegister(2, 'target')
cu2 = QuantumCircuit(c, t)

# WRITE YOUR CODE BETWEEN THESE LINES - START





# WRITE YOUR CODE BETWEEN THESE LINES - END

cu2.draw('mpl')

And you can check your answer below:

In [ ]:
# Check your answer using following code
from qc_grader import grade_ex2b
grade_ex2b(cu2)

Finally, we also need a circuit that is equivalent to applying $U$ four times (i.e. we need the circuit $U^4$).  

<div class="alert alert-block alert-success">
    
**Exercise 2c:** Create a circuit ($U^4$) that performs the transformation:

$$
\begin{aligned}
U|00\rangle &= |00\rangle \\
U|01\rangle &= |01\rangle \\
U|10\rangle &= |10\rangle \\
U|11\rangle &= |11\rangle \\
\end{aligned}
$$

and is controlled by another qubit. The circuit will act on a 2-qubit target register named 'target', and be controlled by another single-qubit register named 'control'. You should assign your finished circuit to the variable '`cu4`'. _Hint: The best solution is very simple._
</div>

In [ ]:
c = QuantumRegister(1, 'control')
t = QuantumRegister(2, 'target')
cu4 = QuantumCircuit(c, t)

# WRITE YOUR CODE BETWEEN THESE LINES - START





# WRITE YOUR CODE BETWEEN THESE LINES - END

cu4.draw('mpl')

You can check your answer using the code below:

In [ ]:
# Check your answer using following code
from qc_grader import grade_ex2c
grade_ex2c(cu4)

<div class="alert alert-block alert-success">

**Exercise 2 final:** Now we have controlled $U$, $U^2$ and $U^4$, we can combine this into a circuit that carries out the quantum part of Shor’s algorithm.

The initialization part is easy: we need to put the counting register into the state $|{+}{+}{+}\rangle$ (which we can do with three H-gates) and we need the target register to be in the state $|1\rangle$ (which we mapped to the computational basis state $|00\rangle$, so we don’t need to do anything here). We'll do all this for you.

_Your_ task is to create a circuit that carries out the controlled-$U$s, that will be used in-between the initialization and the inverse quantum Fourier transform. More formally, we want a circuit:


$$
CU_{c_0 t}CU^2_{c_1 t}CU^4_{c_2 t}
$$

Where $c_0$, $c_1$ and $c_2$ are the three qubits in the ‘counting’ register, $t$ is the ‘target’ register, and $U$ is as <a href="#u-definition">defined in the first part of this exercise</a>. In this notation, $CU_{a b}$ means $CU$ is controlled by $a$ and acts on $b$. An easy solution to this is to simply combine the circuits `cu`, `cu2` and `cu4` that you created above, but you will most likely find a more efficient circuit that has the same behavior!
    
</div>
<div class="alert alert-block alert-danger">
    
Your circuit can only contain [CNOTs](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html) and single qubit [U-gates](https://qiskit.org/documentation/stubs/qiskit.circuit.library.UGate.html). Your score will be the number of CNOTs you use (less is better), as multi-qubit gates are usually much more difficult to carry out on hardware than single-qubit gates. If you're struggling with this requirement, we've included a line of code next to the submission that will convert your circuit to this form, although you're likely to do better by hand.
    
</div>

In [ ]:
# Code to combine your previous solutions into your final submission
cqr = QuantumRegister(3, 'control')
tqr = QuantumRegister(2, 'target')
cux = QuantumCircuit(cqr, tqr)
solutions = [cu, cu2, cu4]
for i in range(3):
    cux = cux.compose(solutions[i], [cqr[i], tqr[0], tqr[1]])
cux.draw('mpl')

In [ ]:
# Check your answer using following code
from qc_grader import grade_ex2_final
# Uncomment the two lines below if you need to convert your circuit to CNOTs and single-qubit gates
#from qiskit import transpile
#cux = transpile(cux, basis_gates=['cx','u'])
grade_ex2_final(cux)

Once you're happy with the circuit, you can submit it below:

In [ ]:
# Submit your answer. You can re-submit at any time.
from qc_grader import submit_ex2_final
submit_ex2_final(cux)

Congratulations! You've finished the exercise. Read on to see your circuit used to factor 35, and see how it performs .

## Using your circuit to factorize 35

The code cell below takes your submission for the exercise and uses it to create a circuit that will give us $\tfrac{s}{r}$, where $s$ is a random integer between $0$ and $r-1$, and $r$ is the period of the function $f(x) = 13^x \bmod 35$.

In [ ]:
from qiskit.circuit.library import QFT
from qiskit import ClassicalRegister
# Create the circuit object
cr = ClassicalRegister(3)
shor_circuit = QuantumCircuit(cqr, tqr, cr)

# Initialise the qubits
shor_circuit.h(cqr)

# Add your circuit
shor_circuit = shor_circuit.compose(cux)

# Perform the inverse QFT and extract the output
shor_circuit.append(QFT(3, inverse=True), cqr)
shor_circuit.measure(cqr, cr)
shor_circuit.draw('mpl')

Let's transpile this circuit and see how large it is, and how many CNOTs it uses:

In [ ]:
from qiskit import Aer, transpile
from qiskit.visualization import plot_histogram
qasm_sim = Aer.get_backend('aer_simulator')
tqc = transpile(shor_circuit, basis_gates=['u', 'cx'], optimization_level=3)
print(f"circuit depth: {tqc.depth()}")
print(f"circuit contains {tqc.count_ops()['cx']} CNOTs")

And let's see what we get:

In [ ]:
counts = qasm_sim.run(tqc).result().get_counts()
plot_histogram(counts)

Assuming everything has worked correctly, we should see equal probability of measuring the numbers $0$, $2$, $4$ and $8$. This is because phase estimation gives us $2^n \cdot \tfrac{s}{r}$, where $n$ is the number of qubits in our counting register (here $n = 3$, $s$ is a random integer between $0$ and $r-1$, and $r$ is the number we're trying to calculate). Let's convert these to fractions that tell us $s/r$ (this is something we can easily calculate classically):

In [ ]:
from fractions import Fraction
n = 3  # n is number of qubits in our 'counting' register
# Cycle through each measurement string
for measurement in counts.keys():
    # Convert the binary string to an 'int', and divide by 2^n
    decimal = int(measurement, 2)/2**n
    # Use the continued fractions algorithm to convert to form a/b
    print(Fraction(decimal).limit_denominator())

We can see the denominator of some of the results will tell us the correct answer $r = 4$. We can verify $r=4$ quickly:

In [ ]:
13**4 % 35

So how do we get the factors from this? There is then a high probability that the greatest common divisor of $N$ and either $a^{r/2}-1$ or $a^{r/2}+1$ is a factor of $N$, and the greatest common divisor is also something we can easily calculate classically.

In [ ]:
from math import gcd # Greatest common divisor
for x in [-1, 1]:
    print(f"Guessed factor: {gcd(13**(4//2)+x, 35)}")

We only need to find one factor, and can use it to divide $N$ to find the other factor. But in this case, _both_ $a^{r/2}-1$ or $a^{r/2}+1$ give us $35$'s factors. We can again verify this is correct:

In [ ]:
7*5

## Running on `ibmq_santiago`

We promised this would run on Santiago, so here we will show you how to do that. In this example we will use a simulated Santiago device for convenience, but you can switch this out for the real device if you want:

In [ ]:
from qiskit.test.mock import FakeSantiago
from qiskit import assemble
from qiskit.visualization import plot_histogram
santiago = FakeSantiago()
real_device = False

## Uncomment this code block to run on the real device
#from qiskit import IBMQ
#IBMQ.load_account()
#provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
#santiago = provider.get_backend('ibmq_santiago')
#real_device = True

# We need to transpile for Santiago
tqc = transpile(shor_circuit, santiago, optimization_level=3)

if not real_device:
    tqc = assemble(tqc)

# Run the circuit and print the counts
counts = santiago.run(tqc).result().get_counts()
plot_histogram(counts)

If your score was low enough, you should see we have a high probability of measuring $0$, $2$, $4$ or $8$ as we saw with the perfect simulation. You will see some extra results due to inaccuracies in the processor and unwanted things interacting with our qubits. This 'noise' gets worse the longer our circuit is, as longer computation time means more time for unwanted interactions, and more gates means more potential errors. This is why we needed to cheat to create the smallest circuit possible.

In the near future, our quantum systems will improve enough that we can start using more advanced error mitigation techniques to overcome these problems, which will mean we can run large enough circuits that we can [perform Shor's algorithm without cheating](https://arxiv.org/pdf/quant-ph/0205095.pdf).

## Additional information

**Created by:** Frank Harkins

**Version:** 1.0.0